In [2]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from unidecode import unidecode

### Defining which columns to keep ###

In [13]:
columns_to_drop_stats = [('Unnamed: 2_level_0', 'Nation'),
                   ('Unnamed: 6_level_0', 'Age'),
                   ('Unnamed: 7_level_0', 'Born'),
                   ('Per 90 Minutes', 'Gls'),
                   ('Per 90 Minutes', 'Ast'),
                   ('Progression', 'PrgP'),
                   ('Playing Time', 'MP'),
                   ('Playing Time', 'Starts'),
                   ('Playing Time', '90s'),
                   ('Per 90 Minutes', 'G+A'),
                   ('Per 90 Minutes', 'G-PK'),
                   ('Per 90 Minutes', 'G+A-PK'),
                   ('Per 90 Minutes', 'xG'),
                   ('Per 90 Minutes', 'xAG'),
                   ('Per 90 Minutes', 'xG+xAG'),
                   ('Unnamed: 5_level_0', 'Comp'),
                   ('Per 90 Minutes', 'npxG'),
                   ('Unnamed: 0_level_0', 'Rk'),
                   ('Per 90 Minutes', 'npxG+xAG'),
                   ('Unnamed: 37_level_0',  'Matches')]

columns_to_drop_passing = [( 'Unnamed: 0_level_0',      'Rk'),
            ( 'Unnamed: 2_level_0',  'Nation'),
            ( 'Unnamed: 3_level_0',     'Pos'),
            ( 'Unnamed: 4_level_0',   'Squad'),
            ( 'Unnamed: 5_level_0',    'Comp'),
            ( 'Unnamed: 6_level_0',     'Age'),
            ( 'Unnamed: 7_level_0',    'Born'),
            ( 'Unnamed: 8_level_0',     '90s'),
            (              'Short',     'Cmp'),
            (              'Short',     'Att'),
            (              'Short',    'Cmp%'),
            (             'Medium',     'Cmp'),
            (             'Medium',     'Att'),
            (             'Medium',    'Cmp%'),
            (               'Long',     'Cmp'),
            (               'Long',     'Att'),
            (               'Long',    'Cmp%'),
            ('Unnamed: 23_level_0',     'Ast'),
            ('Unnamed: 24_level_0',     'xAG'),
            (           'Expected',      'xA'),
            (           'Expected',   'A-xAG'),
            ('Unnamed: 32_level_0', 'Matches')]

columns_to_drop_shooting = [( 'Unnamed: 0_level_0',      'Rk'),
            ( 'Unnamed: 2_level_0',  'Nation'),
            ( 'Unnamed: 3_level_0',     'Pos'),
            ( 'Unnamed: 4_level_0',   'Squad'),
            ( 'Unnamed: 5_level_0',    'Comp'),
            ( 'Unnamed: 6_level_0',     'Age'),
            ( 'Unnamed: 7_level_0',    'Born'),
            ( 'Unnamed: 8_level_0',     '90s'),
            (           'Standard',     'Gls'),
            (           'Standard',   'Sh/90'),
            (           'Standard',  'SoT/90'),
            (           'Standard',    'G/Sh'),
            (           'Standard',   'G/SoT'),
            (           'Standard',      'PK'),
            (           'Standard',   'PKatt'),
            (           'Expected',      'xG'),
            (           'Expected',    'npxG'),
            (           'Expected', 'npxG/Sh'),
            (           'Expected',    'G-xG'),
            (           'Expected', 'np:G-xG'),
            ('Unnamed: 26_level_0', 'Matches')]

columns_to_drop_gca = [( 'Unnamed: 0_level_0',       'Rk'),
            ( 'Unnamed: 2_level_0',   'Nation'),
            ( 'Unnamed: 3_level_0',      'Pos'),
            ( 'Unnamed: 4_level_0',    'Squad'),
            ( 'Unnamed: 5_level_0',     'Comp'),
            ( 'Unnamed: 6_level_0',      'Age'),
            ( 'Unnamed: 7_level_0',     'Born'),
            ( 'Unnamed: 8_level_0',      '90s'),
            (                'SCA',    'SCA90'),
            (          'SCA Types', 'PassDead'),
            (                'GCA',      'GCA'),
            (                'GCA',    'GCA90'),
            (          'GCA Types', 'PassLive'),
            (          'GCA Types', 'PassDead'),
            (          'GCA Types',       'TO'),
            (          'GCA Types',       'Sh'),
            (          'GCA Types',      'Fld'),
            (          'GCA Types',      'Def'),
            ('Unnamed: 25_level_0',  'Matches')]

columns_to_drop_defense = [( 'Unnamed: 0_level_0',      'Rk'),
            ( 'Unnamed: 2_level_0',  'Nation'),
            ( 'Unnamed: 3_level_0',     'Pos'),
            ( 'Unnamed: 4_level_0',   'Squad'),
            ( 'Unnamed: 5_level_0',    'Comp'),
            ( 'Unnamed: 6_level_0',     'Age'),
            ( 'Unnamed: 7_level_0',    'Born'),
            ( 'Unnamed: 8_level_0',     '90s'),
            (         'Challenges',     'Tkl'),
            (         'Challenges',     'Att'),
            (         'Challenges',    'Lost'),
            ('Unnamed: 22_level_0', 'Tkl+Int'),
            ('Unnamed: 24_level_0',     'Err'),
            ('Unnamed: 25_level_0', 'Matches')]

columns_to_drop_possession = [( 'Unnamed: 0_level_0',      'Rk'),
            ( 'Unnamed: 2_level_0',  'Nation'),
            ( 'Unnamed: 3_level_0',     'Pos'),
            ( 'Unnamed: 4_level_0',   'Squad'),
            ( 'Unnamed: 5_level_0',    'Comp'),
            ( 'Unnamed: 6_level_0',     'Age'),
            ( 'Unnamed: 7_level_0',    'Born'),
            ( 'Unnamed: 8_level_0',     '90s'),
            (            'Touches',    'Live'),
            (           'Take-Ons',   'Succ%'),
            (           'Take-Ons',    'Tkld'),
            (           'Take-Ons',   'Tkld%'),
            (            'Carries', 'TotDist'),
            (            'Carries', 'PrgDist'),
            (            'Carries',    'PrgC'),
            (            'Carries',     '1/3'),
            (            'Carries',     'Mis'),
            (            'Carries',     'Dis'),
            ('Unnamed: 31_level_0', 'Matches')]

columns_to_drop_misc = [( 'Unnamed: 0_level_0',      'Rk'),
            ( 'Unnamed: 2_level_0',  'Nation'),
            ( 'Unnamed: 3_level_0',     'Pos'),
            ( 'Unnamed: 4_level_0',   'Squad'),
            ( 'Unnamed: 5_level_0',    'Comp'),
            ( 'Unnamed: 6_level_0',     'Age'),
            ( 'Unnamed: 7_level_0',    'Born'),
            ( 'Unnamed: 8_level_0',     '90s'),
            (        'Performance',    'CrdY'),
            (        'Performance',    'CrdR'),
            (        'Performance',   '2CrdY'),
            (        'Performance',     'Off'),
            (        'Performance',     'Crs'),
            (        'Performance',     'Int'),
            (        'Performance',    'TklW'),
            (        'Performance',   'PKwon'),
            (        'Performance',   'PKcon'),
            (        'Performance',      'OG'),
            (       'Aerial Duels',    'Won%'),
            ('Unnamed: 25_level_0', 'Matches')]

### Actually grabbing the data ###

In [14]:
# Stats
stats_url = 'https://fbref.com/en/comps/Big5/2022-2023/stats/players/2022-2023-Big-5-European-Leagues-Stats'

stats = pd.read_html(stats_url)[0]
stats = stats.drop(columns=columns_to_drop_stats)
stats.columns = [col[1] for col in stats.columns]
stats = stats.drop_duplicates(subset='Player', keep=False)
stats= stats.dropna()
stats = stats[stats['Player'] != 'Player']

# Shooting
shooting_url = 'https://fbref.com/en/comps/Big5/2022-2023/shooting/players/2022-2023-Big-5-European-Leagues-Stats'

shooting = pd.read_html(shooting_url)[0]
shooting = shooting.drop(columns=columns_to_drop_shooting)
shooting.columns = [col[1] for col in shooting.columns]
shooting = shooting.dropna()
shooting = shooting[shooting['Player'] != 'Player']


# Passing
passing_url = 'https://fbref.com/en/comps/Big5/2022-2023/passing/players/2022-2023-Big-5-European-Leagues-Stats'

passing = pd.read_html(passing_url)[0]
passing = passing.drop(columns=columns_to_drop_passing)
passing.columns = [col[1] for col in passing.columns]
passing = passing.dropna()
passing = passing[passing['Player'] != 'Player']


# GCA
gca_url = 'https://fbref.com/en/comps/Big5/2022-2023/gca/players/2022-2023-Big-5-European-Leagues-Stats'

gca = pd.read_html(gca_url)[0]
gca = gca.drop(columns=columns_to_drop_gca)
gca.columns = [col[1] for col in gca.columns]
gca = gca.dropna()
gca = gca[gca['Player'] != 'Player']


# Defense
defense_url = 'https://fbref.com/en/comps/Big5/2022-2023/defense/players/2022-2023-Big-5-European-Leagues-Stats'

defense = pd.read_html(defense_url)[0]
defense = defense.drop(columns=columns_to_drop_defense)
defense.columns = [col[1] for col in defense.columns]
defense = defense.dropna()
defense = defense[defense['Player'] != 'Player']


# Possession (already provided in your original example)
possession_url = 'https://fbref.com/en/comps/Big5/2022-2023/possession/players/2022-2023-Big-5-European-Leagues-Stats'

possession = pd.read_html(possession_url)[0]
possession = possession.drop(columns=columns_to_drop_possession)
possession.columns = [col[1] for col in possession.columns]
possession = possession.dropna()
possession = possession[possession['Player'] != 'Player']


# Misc
misc_url = 'https://fbref.com/en/comps/Big5/2022-2023/misc/players/2022-2023-Big-5-European-Leagues-Stats'

misc = pd.read_html(misc_url)[0]
misc = misc.drop(columns=columns_to_drop_misc)
misc.columns = [col[1] for col in misc.columns]
misc = misc.dropna()
misc = misc[misc['Player'] != 'Player']


# Print the heads of each DataFrame to inspect the data
# print("Shooting Data Head:")
# print(shooting.head())

# print("\nPassing Data Head:")
# print(passing.head())

# print("\nGCA Data Head:")
# print(gca.head())

# print("\nDefense Data Head:")
# print(defense.head())

# print("\nPossession Data Head:")
# print(possession.head())

# print("\nMisc Data Head:")
# print(misc.head())

### Combining into one csv file ###

In [21]:
# Load the data with Dask
defense = defense.astype({'Player': 'object', 'Tkl': 'float32', 'TklW': 'float32', 'Def 3rd': 'float32', 'Mid 3rd': 'float32', 'Att 3rd': 'float32', 'Tkl%': 'float32', 'Blocks': 'float32', 'Sh': 'float32', 'Pass': 'float32', 'Int': 'float32', 'Clr': 'float32'})
gca = gca.astype({'Player': 'object', 'SCA': 'float32', 'PassLive': 'float32', 'TO': 'float32', 'Sh': 'float32', 'Fld': 'float32', 'Def': 'float32'})
misc = misc.astype({'Player': 'object', 'Fls': 'float32', 'Fld': 'float32', 'Recov': 'float32', 'Won': 'float32', 'Lost': 'float32'})
passing = passing.astype({'Player': 'object', 'Cmp': 'float32', 'Att': 'float32', 'Cmp%': 'float32', 'TotDist': 'float32', 'PrgDist': 'float32', 'KP': 'float32', '1/3': 'float32', 'PPA': 'float32', 'CrsPA': 'float32', 'PrgP': 'float32'})
shooting = shooting.astype({'Player': 'object', 'Sh': 'float32', 'SoT': 'float32', 'FK': 'float32', 'SoT%': 'float32', 'Dist': 'float32'})
possession = possession.astype({'Player': 'object', 'Touches': 'float32', 'Def Pen': 'float32', 'Def 3rd': 'float32', 'Mid 3rd': 'float32', 'Att 3rd': 'float32', 'Att Pen': 'float32', 'Att': 'float32', 'Succ': 'float32', 'Carries': 'float32', 'CPA': 'float32', 'Rec': 'float32', 'PrgR': 'float32'})
stats = stats.astype({'Player': 'object', 'Pos': 'object', 'Squad': 'object', 'Min': 'float32', 'Gls': 'float32', 'Ast': 'float32', 'G+A': 'float32', 'G-PK': 'float32', 'PK': 'float32', 'PKatt': 'float32', 'CrdY': 'float32', 'CrdR': 'float32', 'xG': 'float32', 'npxG': 'float32', 'xAG': 'float32', 'npxG+xAG': 'float32', 'PrgC': 'float32', 'PrgR': 'float32'})


# Merge the data on the 'Player' column using an outer join
combined = defense.merge(stats, on='Player', how='inner', suffixes=('', '_stats'))
combined = combined.merge(misc, on='Player', how='inner', suffixes=('', '_misc'))
combined = combined.merge(passing, on='Player', how='inner', suffixes=('', '_pass'))
combined = combined.merge(shooting, on='Player', how='inner', suffixes=('', '_sht'))
combined = combined.merge(gca, on='Player', how='inner', suffixes=('', '_gca'))
combined = combined.merge(possession, on='Player', how='inner', suffixes=('', '_pos'))

### Filtering out extraneous columns we havent yet removed ###

As well as players with less than 900 minutes and any players with NaN values

In [22]:
combined = combined.dropna()

combined = combined.drop(columns=['Def 3rd', 'Mid 3rd', 'Att 3rd', 'TklW', 'Pos', 'G+A', 'G-PK', 'PK', 'PKatt', 'CrdY', 'CrdR', 'xG', 'npxG+xAG', 'Cmp', 'SoT', 'TotDist', 'PrgDist', 'Fld_gca', 'Sh_gca', 'Def Pen', 'Def 3rd_pos', 'Mid 3rd_pos', 'Att 3rd_pos', 'Att Pen', 'PrgR_pos'])

data = combined[combined['Min'] > 900]

### Acquiring the player positions from fotmob.com ###

Using selenium, I searched every player in the database and added their primary position to the data set. Then I manually made adjustments:
- Right Back = Right Wing-Back (and left too)
- Removed Defender and Forward and found real positions

In [23]:
# # Configuration for ChromeDriver (change path as needed)
# service = Service(executable_path='chromedriver-win64/chromedriver.exe')
# chrome_options = Options()
# # Uncomment next line if you want to run Chrome in headless mode
# # chrome_options.add_argument('--headless')
# driver = webdriver.Chrome(service=service, options=chrome_options)

# def get_player_position_fotmob(player_name):
#     # Opening fotmob.com
#     driver.get('https://www.fotmob.com/')
#     wait = WebDriverWait(driver, 10)

#     # Locate the search bar and enter the player name
#     search_box = wait.until(EC.visibility_of_element_located((By.CLASS_NAME, 'react-autosuggest__input')))
#     search_box.clear()
#     search_box.send_keys(player_name)
#     search_box.send_keys(Keys.RETURN)
    
#     # Wait for search results to load and click on the first result
#     try:
#         wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, 'a[href*="/players/"]'))).click()
        
#         # Wait for the player page to load and get the player position
#         position_element = wait.until(EC.visibility_of_element_located((By.CLASS_NAME, 'css-1g41csj-PositionsCSS')))
#         return position_element.text
#     except:
#         return 'Not Found'  # Return 'Not Found' if there's an error

# # Clean up names and fetch positions
# data['Player'] = data['Player'].apply(unidecode)  # Remove accents from names
# data['Position'] = [get_player_position_fotmob(player) for player in data['Player']]

# # Close the browser
# driver.quit()

filtered = pd.read_csv('data/filtered_2223.csv', header=0, usecols=['Player', 'Position'])
data = data.merge(filtered, how='inner', on='Player', suffixes=('', '_positions'))

object_cols = data.select_dtypes(include='object').columns.tolist()
numeric_cols = data.select_dtypes(exclude='object').columns.tolist()

# Reorder the DataFrame columns
data = data[object_cols + numeric_cols]

data.to_csv('data/Top5LeaguesPlayers_2223.csv', index=False)

In [6]:
standings_url = 'https://fbref.com/en/comps/Big5/2022-2023/2022-2023-Big-5-European-Leagues-Stats'

standings = pd.read_html(standings_url)[0]
standings = standings[['Squad', 'LgRk']]
standings.to_csv('data/Top5LeaguesStandings.csv', index=False)


### Scraping Wage Data ###

In [ ]:
# List of URLs
urls = [
    'https://fbref.com/en/comps/9/2022-2023/wages/2022-2023-Premier-League-Wages',
    'https://fbref.com/en/comps/12/2022-2023/wages/2022-2023-La-Liga-Wages',
    'https://fbref.com/en/comps/11/2022-2023/wages/2022-2023-Serie-A-Wages',
    'https://fbref.com/en/comps/20/2022-2023/wages/2022-2023-Bundesliga-Wages',
    'https://fbref.com/en/comps/13/2022-2023/wages/2022-2023-Ligue-1-Wages'
]

# Initialize an empty DataFrame to hold combined data
wages = pd.DataFrame()

# Columns to keep
columns_to_keep = ['Player', 'Weekly Wages']

# Loop over each URL and process the table
for url in urls:
    # Read the table
    tables = pd.read_html(url)
    
    # Assuming the table of interest is the first one
    df = tables[1]
    
    # Keep only the specified columns
    df = df[columns_to_keep]
    
    # Append the data to the combined DataFrame
    wages = pd.concat([wages, df], ignore_index=True)


# Function to extract and convert the weekly wage to float
def extract_weekly_wage(wage):
    match = re.search(r'£ (\d[\d,]*)', wage)
    if match:
        # Remove commas from the matched number and convert to float
        return float(match.group(1).replace(',', ''))
    return None

# Apply the function to the 'Weekly Wages' column
wages['Weekly Wages'] = wages['Weekly Wages'].apply(extract_weekly_wage)

# Display the combined DataFrame
wages.to_csv('data/Top5LeaguesPlayers_Wages.csv', index=False)